# 视频弹幕(中)

In [ ]:
import re
import os
import random

class TrackManager:
    """
    一个智能轨道管理器。
    在新模式下，它的核心职责是为每个在指定时间出现的弹幕，选择一个最佳的Y轴位置（轨道），
    以最大程度地避免视觉上的重叠，但绝不会更改弹幕的出现时间。
    """
    def __init__(self, num_tracks, font_size):
        # track_free_time 现在记录的是“轨道上上一条弹幕的尾巴完全进入屏幕的时间”
        # 这用于防止新弹幕在旧弹幕的尾巴上生成，造成“瞬间重叠”
        self.track_free_time = [0.0] * num_tracks
        self.num_tracks = num_tracks
        self.font_size = font_size

    def _calculate_text_width(self, text):
        width = 0
        for char in text:
            if '\u4e00' <= char <= '\u9fff' or '\u3040' <= char <= '\u30ff' or '\uff00' <= char <= '\uffef':
                width += self.font_size
            else:
                width += self.font_size * 0.6
        return width

    def place_danmaku(self, start_time, text, video_width, danmaku_duration):
        """
        为弹幕分配一个轨道（Y坐标），但绝不改变其开始时间。
        """
        text_width = self._calculate_text_width(text)

        # 动态计算速度，以确保弹幕在固定时长内飘过
        speed = (video_width + text_width) / danmaku_duration
        
        # 计算弹幕的尾巴完全进入屏幕所需的时间，以此作为轨道占用的判断依据
        # 这是为了防止后一条弹幕直接生成在前一条弹幕的“身上”
        time_for_tail_to_enter = text_width / speed
        
        # 寻找在弹幕“出现时”就已经可用的轨道
        available_tracks = [i for i, t in enumerate(self.track_free_time) if t <= start_time]
        
        if available_tracks:
            # 如果有立即可用的轨道，从中随机选一个
            best_track = random.choice(available_tracks)
        else:
            # 如果所有轨道在弹幕出现时都“忙碌”（即上一条弹幕的尾巴还没完全进入屏幕）
            # 我们选择那个最早会变“可用”的轨道。
            # 注意：这不会推迟弹幕的出现，只会让它在视觉上可能与前一条弹幕靠得更近。
            earliest_free_time = min(self.track_free_time)
            best_track = self.track_free_time.index(earliest_free_time)

        # 更新轨道的“下次可用时间”
        self.track_free_time[best_track] = start_time + time_for_tail_to_enter
        
        # 计算Y轴位置
        y_pos = (best_track + 1) * (self.font_size + 8) # 8是轨道间的垂直间距
        
        return y_pos, text_width


def lrc_to_ass_final(lrc_file, ass_file, video_width, video_height, font_size, num_tracks, danmaku_duration):
    header = f"""[Script Info]
Title: Converted from LRC
ScriptType: v4.00+
PlayResX: {video_width}
PlayResY: {video_height}
WrapStyle: 2

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Danmaku,Segoe UI Emoji, KaiTi,{font_size},&H00FFFFFF,&H00FFFFFF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,1.5,0,2,10,10,10,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    lrc_pattern = re.compile(r'\[(\d{2}):(\d{2}):(\d{2})[.:](\d{2,3})\](.*)')

    if not os.path.exists(lrc_file):
        print(f"!!! 错误：找不到弹幕文件 '{lrc_file}'")
        return False

    with open(lrc_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        
    track_manager = TrackManager(num_tracks, font_size)

    for line in lines:
        match = lrc_pattern.search(line)
        if not match: continue
        
        hh, mm, ss, ms_str, raw_text = match.groups()
        ms = int(ms_str.ljust(3, '0'))
        raw_text = raw_text.strip()
        if not raw_text: continue
        
        parts = re.split(r'\s+', raw_text, 1)
        if len(parts) == 2:
            user_id, danmaku_content = parts
            formatted_text = f"{user_id}: {danmaku_content}"
        else:
            formatted_text = raw_text

        # <<< 核心修改 1：严格遵守LRC时间戳 >>>
        start_time = int(hh) * 3600 + int(mm) * 60 + int(ss) + ms / 1000
        # <<< 核心修改 2：使用固定的时长计算结束时间 >>>
        end_time = start_time + danmaku_duration
        
        # 从轨道管理器获取最佳Y坐标和弹幕宽度
        y_pos, text_width = track_manager.place_danmaku(start_time, formatted_text, video_width, danmaku_duration)
        
        # 格式化时间戳
        start_h, start_rem = divmod(start_time, 3600); start_m, start_s = divmod(start_rem, 60)
        start_formatted = f"{int(start_h)}:{int(start_m):02}:{int(start_s):02}.{int((start_s % 1) * 100):02}"
        end_h, end_rem = divmod(end_time, 3600); end_m, end_s = divmod(end_rem, 60)
        end_formatted = f"{int(end_h)}:{int(end_m):02}:{int(end_s):02}.{int((end_s % 1) * 100):02}"
        
        # 创建ASS特效和文本
        effect = f"{{_\\move({video_width}, {y_pos}, {-text_width}, {y_pos})}}"
        final_text = formatted_text.replace('{', '｛').replace('}', '｝')
        events.append(f"Dialogue: 0,{start_formatted},{end_formatted},Danmaku,,0,0,0,,{effect}{final_text}")

    if not events:
        print("\n!!! 严重警告：转换完成，但没有找到任何有效的弹幕行！")
        return False

    with open(ass_file, 'w', encoding='utf-8-sig') as f:
        f.write(header)
        f.write("\n".join(events))
    
    print(f"✅ 成功: LRC 弹幕已转换为 ASS 字幕! 共处理了 {len(events)} 条弹幕。")
    print(f"   文件保存在: {ass_file}\n")
    return True

# --- 主程序 ---
if __name__ == '__main__':
    video_full_path = r"D:\48tool\pocket48\pocket48-live-broadcast-replay\[口袋48录播]_CGT48-谭思慧_☁️_2025-07-03~17.37.29_1147586109335474176__2025-07-05~04.24.30.ts"
    lrc_full_path = r"D:\48tool\pocket48\pocket48-live-broadcast-replay\[口袋48弹幕]_CGT48-谭思慧_☁️_2025-07-03~17.37.29_1147586109335474176__2025-07-05~04.24.40.lrc"
    
    # --- 在这里精细化配置你的参数 ---
    
    # <<< 全新核心参数：弹幕持续时长 (秒) >>>
    # 这个值决定了每条弹幕从出现到消失需要多长时间。
    # 推荐值: 8 到 12 之间。值越大，弹幕移动越慢；值越小，移动越快。
    # 这将确保所有弹幕的显示时间都是固定的，解决了“时间变长”的问题。
    danmaku_duration = 8

    num_tracks = 15      # 弹幕轨道数量
    font_size = 25       # 弹幕字体大小

    output_directory = os.path.dirname(lrc_full_path)
    base_filename = os.path.splitext(os.path.basename(lrc_full_path))[0]
    
    ass_full_path = os.path.join(output_directory, base_filename + ".ass")
    temp_mp4_path = os.path.join(output_directory, base_filename + "_video_only.mp4")
    final_mp4_path = os.path.join(output_directory, base_filename + "_带弹幕.mp4")

    print(f"--- 持续时长:{danmaku_duration}s | 轨道:{num_tracks} | 字体:{font_size}px ---")
    print("--- 第一步：正在转换LRC为ASS字幕... ---")
    
    # 注意：lrc_to_ass_final的参数已更新
    if lrc_to_ass_final(lrc_full_path, ass_file=ass_full_path, video_width=1920, video_height=1080, font_size=font_size, num_tracks=num_tracks, danmaku_duration=danmaku_duration):
        print("--- 第二步：请打开命令行，按顺序复制并执行以下FFmpeg命令 ---")
        
        print("\n【命令 1: (推荐) 将TS无损转换为MP4】")
        command1 = f'ffmpeg -i "{video_full_path}" -c copy -y "{temp_mp4_path}"'
        print("复制此行 ->", command1)
        
        print("\n【命令 2: (最终) 合并视频和字幕为软字幕MP4】")
        command2 = f'ffmpeg -i "{temp_mp4_path}" -i "{ass_full_path}" -c copy -c:s mov_text -y "{final_mp4_path}"'
        print("复制此行 ->", command2)
        
        print(f"\n🎉 全部完成后，你的最终带弹幕视频在: {final_mp4_path}")

# 口袋48直播风格软弹幕(下)

In [ ]:
import re
import os

def lrc_to_ass_chatbox_region(lrc_file, ass_file, video_width, video_height, font_size, line_spacing, chatbox_max_height_ratio, chatbox_duration_after_last):
    """
    将LRC文件转换为抖音/直播风格的左下角“固定区域”滚动弹幕框ASS字幕。
    这个版本针对竖屏视频进行了最终优化，并强制左下角对齐。
    """
    
    # --- ASS字幕头部和样式定义 ---
    # 调整了描边和阴影，以适应更小的字体，增强可读性。
    header = f"""[Script Info]
Title: Converted from LRC to Chatbox
ScriptType: v4.00+
PlayResX: {video_width}
PlayResY: {video_height}
WrapStyle: 2

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Chatbox,Segoe UI Emoji, Microsoft YaHei,{font_size},&H00FFFFFF,&H00FFFFFF,&H00000000,&H99000000,-1,0,0,0,100,100,0,0,3,1,0.5,1,15,15,15,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    
    # --- 1. 读取并解析所有LRC行 ---
    lrc_pattern = re.compile(r'\[(\d{2}):(\d{2}):(\d{2})[.:](\d{2,3})\](.*)')
    if not os.path.exists(lrc_file):
        print(f"!!! 错误：找不到弹幕文件 '{lrc_file}'")
        return False

    all_comments = []
    with open(lrc_file, 'r', encoding='utf-8') as f:
        for line in f:
            match = lrc_pattern.search(line)
            if not match: continue
            
            hh, mm, ss, ms_str, raw_text = match.groups()
            ms = int(ms_str.ljust(3, '0'))
            raw_text = raw_text.strip()
            if not raw_text: continue
            
            parts = re.split(r'\s+', raw_text, 1)
            if len(parts) == 2:
                user_id, danmaku_content = parts
                formatted_text = f"{{\\c&H00FFFF&}}{user_id}:{{\\c&HFFFFFF&}} {danmaku_content}"
            else:
                formatted_text = raw_text
                
            start_time = int(hh) * 3600 + int(mm) * 60 + int(ss) + ms / 1000
            all_comments.append((start_time, formatted_text))

    if not all_comments:
        print("\n!!! 严重警告：转换完成，但没有找到任何有效的弹幕行！")
        return False
        
    all_comments.sort(key=lambda x: x[0])

    # --- 2. 基于“固定区域”逻辑生成ASS事件 ---
    events = []
    max_pixel_height = video_height * chatbox_max_height_ratio
    line_height = font_size + line_spacing

    for i in range(len(all_comments)):
        current_time = all_comments[i][0]
        
        if i + 1 < len(all_comments):
            end_time = all_comments[i+1][0]
        else:
            end_time = current_time + chatbox_duration_after_last
        
        if end_time - current_time < 0.1:
            continue

        lines_for_this_frame = []
        current_height = 0
        for j in range(i, -1, -1):
            comment_to_add = all_comments[j][1]
            if current_height + line_height > max_pixel_height:
                break
            lines_for_this_frame.insert(0, comment_to_add)
            current_height += line_height
        
        text_block = "\\N".join(lines_for_this_frame)
        
        # <<< --- 决定性修复：在文本块前强制加上 {\an1} 标签 --- >>>
        final_text_block = f"{{\\an1}}{text_block}"
        
        start_h, start_rem = divmod(current_time, 3600); start_m, start_s = divmod(start_rem, 60)
        start_formatted = f"{int(start_h)}:{int(start_m):02}:{int(start_s):02}.{int((start_s % 1) * 100):02}"
        end_h, end_rem = divmod(end_time, 3600); end_m, end_s = divmod(end_rem, 60)
        end_formatted = f"{int(end_h)}:{int(end_m):02}:{int(end_s):02}.{int((end_s % 1) * 100):02}"
        
        events.append(f"Dialogue: 0,{start_formatted},{end_formatted},Chatbox,,0,0,0,,{final_text_block}")

    # --- 3. 写入文件 ---
    with open(ass_file, 'w', encoding='utf-8-sig') as f:
        f.write(header)
        f.write("\n".join(events))
    
    print(f"✅ 成功: LRC 弹幕已转换为 Chatbox (固定区域) 风格的 ASS 字幕! 共生成 {len(events)} 个显示事件。")
    print(f"   文件保存在: {ass_file}\n")
    return True

# --- 主程序 ---
if __name__ == '__main__':
    video_full_path = r"D:\48tool\pocket48\pocket48-live-broadcast-replay\[口袋48录播]_CGT48-谭思慧_☁️_2025-07-03~23.45.28_1147678717646802944__2025-07-04~19.07.51.ts"
    lrc_full_path = r"D:\48tool\pocket48\pocket48-live-broadcast-replay\[口袋48弹幕]_CGT48-谭思慧_☁️_2025-07-03~23.45.28_1147678717646802944__2025-07-04~18.38.50.lrc"
    
    # --- 在这里精确配置你的参数 ---
    
    # 视频的实际分辨率
    video_width = 540
    video_height = 960
    
    # 字体大小 (已按您的建议改为更小的值)
    font_size = 5
    
    # 弹幕框最大高度占屏幕高度的比例 (0.3 = 30%)
    chatbox_max_height_ratio = 0.3
    
    # 行间距（像素），配合小字体，行距也应减小
    line_spacing = 2
    
    # 最后一条弹幕显示多久（秒）
    chatbox_duration_after_last = 10 

    # --- 自动生成输出文件路径 ---
    output_directory = os.path.dirname(lrc_full_path)
    base_filename = os.path.splitext(os.path.basename(lrc_full_path))[0]
    
    ass_full_path = os.path.join(output_directory, base_filename + "_chatbox_final.ass")
    temp_mp4_path = os.path.join(output_directory, base_filename + "_video_only.mp4")
    final_mp4_path = os.path.join(output_directory, base_filename + "_带弹幕(chatbox_final).mp4")

    print(f"--- 模式:固定区域弹幕框 | 视频尺寸:{video_width}x{video_height} | 字体:{font_size}px ---")
    print("--- 第一步：正在转换LRC为ASS字幕... ---")
    
    if lrc_to_ass_chatbox_region(
        lrc_file=lrc_full_path, 
        ass_file=ass_full_path, 
        video_width=video_width, 
        video_height=video_height, 
        font_size=font_size, 
        line_spacing=line_spacing,
        chatbox_max_height_ratio=chatbox_max_height_ratio,
        chatbox_duration_after_last=chatbox_duration_after_last
    ):
        print("--- 第二步：请打开命令行，按顺序复制并执行以下FFmpeg命令 ---")
        
        print("\n【命令 1: (推荐) 将TS无损转换为MP4】")
        command1 = f'ffmpeg -i "{video_full_path}" -c copy -y "{temp_mp4_path}"'
        print("复制此行 ->", command1)
        
        print("\n【命令 2: (最终) 合并视频和字幕为软字幕MP4】")
        command2 = f'ffmpeg -i "{temp_mp4_path}" -i "{ass_full_path}" -c copy -c:s mov_text -y "{final_mp4_path}"'
        print("复制此行 ->", command2)
        
        print(f"\n🎉 全部完成后，你的最终带弹幕视频在: {final_mp4_path}")

# 口袋48直播硬弹幕(b站版)

In [4]:
import re
import os

def wrap_text(text, width=20, indent="  "):
    """
    一个辅助函数，用于将长文本按指定宽度换行。
    后续行会自动添加缩进以提高可读性。
    :param text: 需要处理的原始文本。
    :param width: 每行的最大宽度（字符数）。
    :param indent: 后续换行的缩进字符串。
    :return: 一个处理过的、可能包含ASS换行符 `\\N` 的字符串。
    """
    if len(text) <= width:
        return text
    
    lines = []
    # 处理第一行，不加缩进
    lines.append(text[:width])
    
    # 循环处理剩余的文本
    remaining_text = text[width:]
    # 后续行的有效宽度需要减去缩进的长度
    sub_width = width - len(indent)

    while remaining_text:
        # 取出一个块
        chunk = remaining_text[:sub_width]
        # 加上缩进后存入列表
        lines.append(indent + chunk)
        # 更新剩余文本
        remaining_text = remaining_text[sub_width:]
    
    # 使用ASS的换行符 `\N` 将所有行连接起来
    return "\\N".join(lines)

def lrc_to_ass_chatbox_region(
    lrc_file, ass_file, video_width, video_height,
    font_name, font_size, line_spacing, letter_spacing, chatbox_max_height_ratio,
    margin_left, margin_bottom, chatbox_duration_after_last, wrap_width
):
    """
    【已修正：实现弹幕间空行效果】
    将 LRC 转 ASS：左下角固定区域弹幕框，使用绝对定位和剪裁确保字体尺寸及高度限制生效
    """
    # 根据视频高度和比例，计算出弹幕区域的最大像素高度
    max_pixel_height = int(video_height * chatbox_max_height_ratio)

    # --- 1. ASS文件头部和样式定义 ---
    header = f"""[Script Info]
Title: Converted from LRC to Chatbox
ScriptType: v4.00+
PlayResX: {video_width}
PlayResY: {video_height}
WrapStyle: 2

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Chatbox,{font_name},{font_size},&H00FFFFFF,&H00FFFFFF,&H00000000,&H00000000,0,0,0,0,100,100,{letter_spacing},0,1,0,0,1,0,0,{line_spacing},1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
#  【↑↑↑ 已修改 ↑↑↑】
#  1. BackColour (背景色): 从 &H99000000 (半透明) 改为 &H00000000 (完全不透明)。
#  2. Outline (描边): 从 1 改为 0。

    # --- 2. 读取并解析LRC文件 ---
    if not os.path.exists(lrc_file):
        print(f"!!! 错误：找不到弹幕文件 '{lrc_file}'"); return False
    
    try:
        with open(lrc_file, 'r', encoding='utf-8-sig') as f:
            lrc_content = f.readlines()
    except Exception as e:
        print(f"!!! 错误：读取LRC文件失败: {e}"); return False

    pattern = re.compile(r'\[(\d{2}):(\d{2}):(\d{2})[.:](\d{2,3})\](.*)')
    comments = []
    for line in lrc_content:
        m = pattern.search(line)
        if not m: continue
        hh, mm, ss, ms_s, txt = m.groups()
        ms = int(ms_s.ljust(3, '0'))
        txt = txt.strip()
        if not txt: continue
        
        parts = re.split(r'\s+', txt, 1)
        formatted_txt = f"{parts[0]}:{parts[1]}" if len(parts) == 2 else txt
        
        wrapped_txt = wrap_text(formatted_txt, width=wrap_width)
        
        t = int(hh) * 3600 + int(mm) * 60 + int(ss) + ms / 1000
        comments.append((t, wrapped_txt)) 
        
    if not comments:
        print("!!! 警告：未解析到任何弹幕！"); return False
    comments.sort(key=lambda x: x[0])

    # --- 3. 生成ASS事件 ---
    events = []
    # 计算每一“逻辑行”的高度，包括字体大小和额外的行间距
    line_h = font_size + line_spacing
    pos_x = margin_left
    pos_y = video_height - margin_bottom

    def fmt_time(t):
        h = int(t // 3600); m = int((t - h * 3600) // 60); s = t - h * 3600 - m * 60
        cs = int((s - int(s)) * 100); return f"{h}:{m:02}:{int(s):02}.{cs:02}"

    for i, (start_t, _) in enumerate(comments):
        end_t = comments[i+1][0] if i+1 < len(comments) else start_t + chatbox_duration_after_last
        if end_t - start_t < 0.1: continue
        
        h_acc = 0  # 累积高度
        lines_to_display = []
        # 从当前弹幕开始，倒序收集弹幕
        for j in range(i, -1, -1):
            comment_text = comments[j][1]
            
            # --- 【关键改动 1: 高度计算】 ---
            # 计算当前这条弹幕文本自身的高度
            comment_height = (comment_text.count('\\N') + 1) * line_h
            
            # 计算分隔符（即那个空行）的高度
            # 只有当显示列表中已经有弹幕时，才需要在新弹幕顶部添加一个空行
            separator_height = line_h if lines_to_display else 0
            
            # 检查：如果把“这条弹幕”和它上面的“空行”一起加进去，是否会超出最大高度？
            if h_acc + comment_height + separator_height > max_pixel_height:
                break # 如果会超出，就停止收集
            
            # 如果没问题，就把这条弹幕和它上面的空行的高度都加到累积高度中
            h_acc += comment_height + separator_height
            
            # 将弹幕文本插入到显示列表的开头
            lines_to_display.insert(0, comment_text)

        # --- 【关键改动 2: 文本连接】 ---
        # 使用双换行符 `\\N\\N` 来连接所有要显示的弹幕，以创造空行
        text = "\\N\\N".join(lines_to_display)
        
        x1, y1 = margin_left, pos_y - max_pixel_height; x2, y2 = video_width - margin_left, pos_y
        override = (f"{{\\an1\\pos({pos_x},{pos_y})\\fs{font_size}\\fn{font_name}\\fsp{letter_spacing}\\clip({x1},{y1},{x2},{y2})}}")
        full_text = f"{override}{text}"
        events.append(f"Dialogue: 0,{fmt_time(start_t)},{fmt_time(end_t)},Chatbox,,0,0,0,,{full_text}")

    # --- 4. 写入ASS文件 ---
    try:
        with open(ass_file, 'w', encoding='utf-8-sig') as fw:
            fw.write(header); fw.write("\n".join(events))
    except Exception as e:
        print(f"!!! 错误：写入ASS文件失败: {e}"); return False
    print(f"✅ 成功生成 {len(events)} 条弹幕事件，已保存到: {ass_file}"); return True

# --- 主程序 ---
if __name__ == '__main__':
    # =========================================================================
    # 1. 路径配置 (请确保文件路径正确)
    # =========================================================================
    # !! 请在这里修改为你自己的文件路径 !!
    lrc_full_path = r"E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\B站\[口袋48弹幕]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27.lrc"
    ts_full_path = r"E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\B站\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05.ts"
    
    # -------------------------------------------------------------------------
    # 2. 自动生成输出文件名 (通常无需修改)
    # -------------------------------------------------------------------------
    ass_full_path = os.path.splitext(lrc_full_path)[0] + "_chatbox_fixed.ass"
    mp4_final_path = os.path.splitext(ts_full_path)[0] + "_chatbox_hardsub_final.mp4"

    # =========================================================================
    # 3. 视频参数配置 (所有可调整的参数都在这里！)
    # =========================================================================
    
    # --- 视频基本信息 ---
    video_width = 540                 # 视频的原始宽度（像素）。
    video_height = 960                # 视频的原始高度（像素）。

    # --- 字体与样式 ---
    font_name = "楷体"                # 要使用的字体名称。
    font_size = 18                    # 弹幕的字体大小（磅）。
    line_spacing = 0                 # 弹幕行与行之间的额外垂直间距（像素）。
    letter_spacing = 0                # 每个文字之间的额外水平间距（像素）。

    # --- 自动换行 ---
    wrap_width = 18                   # 自动换行的宽度，单位是字符数。

    # --- 布局与定位 ---
    chatbox_max_height_ratio = 0.25   # 弹幕区域允许占用的最大高度，占视频总高度的百分比。
    margin_left = 40                  # 弹幕区域距离视频左侧边缘的距离（像素）。
    margin_bottom = 198               # 弹幕区域底部距离视频底部边缘的距离（像素）。

    # --- 时间控制 ---
    chatbox_duration_after_last = 10  # 最后一条弹幕出现后，弹幕框继续显示的秒数。

    # =========================================================================
    # 4. 执行LRC到ASS的转换 (无需修改)
    # =========================================================================
    success = lrc_to_ass_chatbox_region(
        lrc_file=lrc_full_path, ass_file=ass_full_path, video_width=video_width,
        video_height=video_height, font_name=font_name, font_size=font_size,
        line_spacing=line_spacing, letter_spacing=letter_spacing,
        chatbox_max_height_ratio=chatbox_max_height_ratio,
        margin_left=margin_left, margin_bottom=margin_bottom,
        chatbox_duration_after_last=chatbox_duration_after_last,
        wrap_width=wrap_width
    )
    
    # =========================================================================
    # 5. 生成FFmpeg命令 (无需修改)
    # =========================================================================
    if success:
        if not os.path.exists(ts_full_path):
            print("\n" + "!"*80); print(f"!!! 严重警告：找不到指定的视频文件！路径：'{ts_full_path}'"); print("!"*80 + "\n")
        else:
            print("\n" + "="*80); print("🎉 ASS字幕已生成！请使用以下命令压制【视觉无损】硬字幕视频："); print("="*80 + "\n")
            escaped_ass_path = ass_full_path.replace('\\', '/').replace(':', '\\:').replace('[', '\\[').replace(']', '\\]')
            
            ffmpeg_command = (
                f'ffmpeg -i "{ts_full_path}" '
                f'-vf "ass=filename=\'{escaped_ass_path}\'" '
                f'-c:v libx264 '
                f'-preset veryfast '
                f'-crf 15 '
                f'-c:a aac '
                f'-y "{mp4_final_path}"'
            )
            
            print("【视觉无损硬字幕压制命令 (Visually Lossless)】")
            print("-" * 50)
            print(ffmpeg_command)
            print("-" * 50)
            print("\n请复制以上命令，在终端中粘贴并运行。\n")

✅ 成功生成 1003 条弹幕事件，已保存到: E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\B站\[口袋48弹幕]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27_chatbox_fixed.ass

🎉 ASS字幕已生成！请使用以下命令压制【视觉无损】硬字幕视频：

【视觉无损硬字幕压制命令 (Visually Lossless)】
--------------------------------------------------
ffmpeg -i "E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\B站\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05.ts" -vf "ass=filename='E\:/48tool/pocket48/pocket48-live-broadcast-replay/20250706/B站/\[口袋48弹幕\]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27_chatbox_fixed.ass'" -c:v libx264 -preset veryfast -crf 15 -c:a aac -y "E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\B站\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05_chatbox_hardsub_final.mp4"
--------------------------------------------------

请复制以上命令，在终端中粘贴并运行。



# 口袋48直播硬弹幕微博版)

In [5]:
import re
import os

def wrap_text(text, width=20, indent="  "):
    """
    一个辅助函数，用于将长文本按指定宽度换行。
    后续行会自动添加缩进以提高可读性。
    :param text: 需要处理的原始文本。
    :param width: 每行的最大宽度（字符数）。
    :param indent: 后续换行的缩进字符串。
    :return: 一个处理过的、可能包含ASS换行符 `\\N` 的字符串。
    """
    if len(text) <= width:
        return text
    
    lines = []
    # 处理第一行，不加缩进
    lines.append(text[:width])
    
    # 循环处理剩余的文本
    remaining_text = text[width:]
    # 后续行的有效宽度需要减去缩进的长度
    sub_width = width - len(indent)

    while remaining_text:
        # 取出一个块
        chunk = remaining_text[:sub_width]
        # 加上缩进后存入列表
        lines.append(indent + chunk)
        # 更新剩余文本
        remaining_text = remaining_text[sub_width:]
    
    # 使用ASS的换行符 `\N` 将所有行连接起来
    return "\\N".join(lines)

def lrc_to_ass_chatbox_region(
    lrc_file, ass_file, video_width, video_height,
    font_name, font_size, line_spacing, letter_spacing, chatbox_max_height_ratio,
    margin_left, margin_bottom, chatbox_duration_after_last, wrap_width
):
    """
    【已修正：实现弹幕间空行效果】
    将 LRC 转 ASS：左下角固定区域弹幕框，使用绝对定位和剪裁确保字体尺寸及高度限制生效
    """
    # 根据视频高度和比例，计算出弹幕区域的最大像素高度
    max_pixel_height = int(video_height * chatbox_max_height_ratio)

    # --- 1. ASS文件头部和样式定义 ---
    header = f"""[Script Info]
Title: Converted from LRC to Chatbox
ScriptType: v4.00+
PlayResX: {video_width}
PlayResY: {video_height}
WrapStyle: 2

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Chatbox,{font_name},{font_size},&H00FFFFFF,&H00FFFFFF,&H00000000,&H00000000,0,0,0,0,100,100,{letter_spacing},0,1,0,0,1,0,0,{line_spacing},1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
#  【↑↑↑ 已修改 ↑↑↑】
#  1. BackColour (背景色): 从 &H99000000 (半透明) 改为 &H00000000 (完全不透明)。
#  2. Outline (描边): 从 1 改为 0。

    # --- 2. 读取并解析LRC文件 ---
    if not os.path.exists(lrc_file):
        print(f"!!! 错误：找不到弹幕文件 '{lrc_file}'"); return False
    
    try:
        with open(lrc_file, 'r', encoding='utf-8-sig') as f:
            lrc_content = f.readlines()
    except Exception as e:
        print(f"!!! 错误：读取LRC文件失败: {e}"); return False

    pattern = re.compile(r'\[(\d{2}):(\d{2}):(\d{2})[.:](\d{2,3})\](.*)')
    comments = []
    for line in lrc_content:
        m = pattern.search(line)
        if not m: continue
        hh, mm, ss, ms_s, txt = m.groups()
        ms = int(ms_s.ljust(3, '0'))
        txt = txt.strip()
        if not txt: continue
        
        parts = re.split(r'\s+', txt, 1)
        formatted_txt = f"{parts[0]}:{parts[1]}" if len(parts) == 2 else txt
        
        wrapped_txt = wrap_text(formatted_txt, width=wrap_width)
        
        t = int(hh) * 3600 + int(mm) * 60 + int(ss) + ms / 1000
        comments.append((t, wrapped_txt)) 
        
    if not comments:
        print("!!! 警告：未解析到任何弹幕！"); return False
    comments.sort(key=lambda x: x[0])

    # --- 3. 生成ASS事件 ---
    events = []
    # 计算每一“逻辑行”的高度，包括字体大小和额外的行间距
    line_h = font_size + line_spacing
    pos_x = margin_left
    pos_y = video_height - margin_bottom

    def fmt_time(t):
        h = int(t // 3600); m = int((t - h * 3600) // 60); s = t - h * 3600 - m * 60
        cs = int((s - int(s)) * 100); return f"{h}:{m:02}:{int(s):02}.{cs:02}"

    for i, (start_t, _) in enumerate(comments):
        end_t = comments[i+1][0] if i+1 < len(comments) else start_t + chatbox_duration_after_last
        if end_t - start_t < 0.1: continue
        
        h_acc = 0  # 累积高度
        lines_to_display = []
        # 从当前弹幕开始，倒序收集弹幕
        for j in range(i, -1, -1):
            comment_text = comments[j][1]
            
            # --- 【关键改动 1: 高度计算】 ---
            # 计算当前这条弹幕文本自身的高度
            comment_height = (comment_text.count('\\N') + 1) * line_h
            
            # 计算分隔符（即那个空行）的高度
            # 只有当显示列表中已经有弹幕时，才需要在新弹幕顶部添加一个空行
            separator_height = line_h if lines_to_display else 0
            
            # 检查：如果把“这条弹幕”和它上面的“空行”一起加进去，是否会超出最大高度？
            if h_acc + comment_height + separator_height > max_pixel_height:
                break # 如果会超出，就停止收集
            
            # 如果没问题，就把这条弹幕和它上面的空行的高度都加到累积高度中
            h_acc += comment_height + separator_height
            
            # 将弹幕文本插入到显示列表的开头
            lines_to_display.insert(0, comment_text)

        # --- 【关键改动 2: 文本连接】 ---
        # 使用双换行符 `\\N\\N` 来连接所有要显示的弹幕，以创造空行
        text = "\\N\\N".join(lines_to_display)
        
        x1, y1 = margin_left, pos_y - max_pixel_height; x2, y2 = video_width - margin_left, pos_y
        override = (f"{{\\an1\\pos({pos_x},{pos_y})\\fs{font_size}\\fn{font_name}\\fsp{letter_spacing}\\clip({x1},{y1},{x2},{y2})}}")
        full_text = f"{override}{text}"
        events.append(f"Dialogue: 0,{fmt_time(start_t)},{fmt_time(end_t)},Chatbox,,0,0,0,,{full_text}")

    # --- 4. 写入ASS文件 ---
    try:
        with open(ass_file, 'w', encoding='utf-8-sig') as fw:
            fw.write(header); fw.write("\n".join(events))
    except Exception as e:
        print(f"!!! 错误：写入ASS文件失败: {e}"); return False
    print(f"✅ 成功生成 {len(events)} 条弹幕事件，已保存到: {ass_file}"); return True

# --- 主程序 ---
if __name__ == '__main__':
    # =========================================================================
    # 1. 路径配置 (请确保文件路径正确)
    # =========================================================================
    # !! 请在这里修改为你自己的文件路径 !!
    lrc_full_path = r"E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\微博\[口袋48弹幕]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27.lrc"
    ts_full_path = r"E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\微博\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05.ts"
    
    # -------------------------------------------------------------------------
    # 2. 自动生成输出文件名 (通常无需修改)
    # -------------------------------------------------------------------------
    ass_full_path = os.path.splitext(lrc_full_path)[0] + "_chatbox_fixed.ass"
    mp4_final_path = os.path.splitext(ts_full_path)[0] + "_chatbox_hardsub_final.mp4"

    # =========================================================================
    # 3. 视频参数配置 (所有可调整的参数都在这里！)
    # =========================================================================
    
    # --- 视频基本信息 ---
    video_width = 540                 # 视频的原始宽度（像素）。
    video_height = 960                # 视频的原始高度（像素）。

    # --- 字体与样式 ---
    font_name = "楷体"                # 要使用的字体名称。
    font_size = 18                    # 弹幕的字体大小（磅）。
    line_spacing = 0                 # 弹幕行与行之间的额外垂直间距（像素）。
    letter_spacing = 0                # 每个文字之间的额外水平间距（像素）。

    # --- 自动换行 ---
    wrap_width = 18                   # 自动换行的宽度，单位是字符数。

    # --- 布局与定位 ---
    chatbox_max_height_ratio = 0.22   # 弹幕区域允许占用的最大高度，占视频总高度的百分比。
    margin_left = 50                  # 弹幕区域距离视频左侧边缘的距离（像素）。
    margin_bottom = 190               # 弹幕区域底部距离视频底部边缘的距离（像素）。

    # --- 时间控制 ---
    chatbox_duration_after_last = 10  # 最后一条弹幕出现后，弹幕框继续显示的秒数。

    # =========================================================================
    # 4. 执行LRC到ASS的转换 (无需修改)
    # =========================================================================
    success = lrc_to_ass_chatbox_region(
        lrc_file=lrc_full_path, ass_file=ass_full_path, video_width=video_width,
        video_height=video_height, font_name=font_name, font_size=font_size,
        line_spacing=line_spacing, letter_spacing=letter_spacing,
        chatbox_max_height_ratio=chatbox_max_height_ratio,
        margin_left=margin_left, margin_bottom=margin_bottom,
        chatbox_duration_after_last=chatbox_duration_after_last,
        wrap_width=wrap_width
    )
    
    # =========================================================================
    # 5. 生成FFmpeg命令 (无需修改)
    # =========================================================================
    if success:
        if not os.path.exists(ts_full_path):
            print("\n" + "!"*80); print(f"!!! 严重警告：找不到指定的视频文件！路径：'{ts_full_path}'"); print("!"*80 + "\n")
        else:
            print("\n" + "="*80); print("🎉 ASS字幕已生成！请使用以下命令压制【视觉无损】硬字幕视频："); print("="*80 + "\n")
            escaped_ass_path = ass_full_path.replace('\\', '/').replace(':', '\\:').replace('[', '\\[').replace(']', '\\]')
            
            ffmpeg_command = (
                f'ffmpeg -i "{ts_full_path}" '
                f'-vf "ass=filename=\'{escaped_ass_path}\'" '
                f'-c:v libx264 '
                f'-preset veryfast '
                f'-crf 15 '
                f'-c:a aac '
                f'-y "{mp4_final_path}"'
            )
            
            print("【视觉无损硬字幕压制命令 (Visually Lossless)】")
            print("-" * 50)
            print(ffmpeg_command)
            print("-" * 50)
            print("\n请复制以上命令，在终端中粘贴并运行。\n")

✅ 成功生成 1003 条弹幕事件，已保存到: E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\微博\[口袋48弹幕]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27_chatbox_fixed.ass

🎉 ASS字幕已生成！请使用以下命令压制【视觉无损】硬字幕视频：

【视觉无损硬字幕压制命令 (Visually Lossless)】
--------------------------------------------------
ffmpeg -i "E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\微博\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05.ts" -vf "ass=filename='E\:/48tool/pocket48/pocket48-live-broadcast-replay/20250706/微博/\[口袋48弹幕\]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.27_chatbox_fixed.ass'" -c:v libx264 -preset veryfast -crf 15 -c:a aac -y "E:\48tool\pocket48\pocket48-live-broadcast-replay\20250706\微博\[口袋48录播]_CGT48-谭思慧_˗ˏˋ ♡ ˎˊ˗_2025-07-06~23.01.16_1148754757081174016__2025-07-07~00.54.05_chatbox_hardsub_final.mp4"
--------------------------------------------------

请复制以上命令，在终端中粘贴并运行。

